In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import gc
import json
from datetime import datetime
import time

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import imagenet_utils

from sklearn.metrics import fbeta_score, precision_score, recall_score

from utils.loaderjpg import *
from utils.generator import *
from utils.visualization import *
from utils.file import *

from pretrained.vgg16 import *
from pretrained.vgg19 import *
from pretrained.resnet50 import *
from pretrained.densenet121 import *
from pretrained.inceptionv3 import *

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
data_dir = 'D:/Downloads/amazon/'

rescaled_dim = 224

#model_name = 'vgg16'
model_name = 'vgg19'
#model_name = 'resnet50'
#model_name = 'densenet121'
#model_name = 'inceptionv3'

In [3]:
if not is_training_set_in_cache(rescaled_dim):
    # populate the training dataset cache
    df_train = pd.read_csv(data_dir + 'train_v2.csv')
    load_training_set(df_train, rescaled_dim)

In [4]:
# if not is_test_set_in_cache(rescaled_dim):
#     # populate the test dataset cache
#     df_test = pd.read_csv(data_dir + 'sample_submission_v2.csv')
#     load_test_set(df_test, rescaled_dim)

In [7]:
from utils.generator import *

bottleneck_weights_file = data_dir + 'bottleneck/{}/bottleneck_features.npy'.format(model_name)

if not os.path.exists(bottleneck_weights_file):
    df_train = pd.read_csv(data_dir + 'train_v2.csv')
    x_train, y_train = load_training_set(df_train, rescaled_dim)
    print(x_train.shape)
    print(y_train.shape)

    if model_name == 'vgg16':
        model = vgg16_model_fc_truncated(channel=3)
    elif model_name == 'vgg19':
        model = vgg19_model_fc_truncated(channel=3)
    elif model_name == 'resnet50':
        model = applications.ResNet50(weights='imagenet', include_top=False)
    elif model_name == 'densenet121':
        model = densenet121_fc_truncated(rescaled_dim, rescaled_dim, color_type=3, num_classes=17)
    elif model_name == 'inceptionv3':
        model = applications.InceptionV3(weights='imagenet', include_top=False)
    else:
        raise ValueError('ERROR: unsupport model {}'.format(model_name))

    img_normalization = image_normalization_func(model_name)
        
    train_datagen = BottleNeckImgGenerator(normalization = img_normalization)
    train_gen = train_datagen.bottleNeckGen(x_train, 64)
    
    bottleneck_features = model.predict_generator(train_gen, x_train.shape[0])
    np.save(bottleneck_weights_file, bottleneck_features)

In [8]:
# print(bottleneck_features.shape)
# print(bottleneck_features[1])

In [9]:
x_train = np.load(bottleneck_weights_file)

df_train = pd.read_csv(data_dir + 'train_v2.csv')
y_train = load_training_set_y_from_cache(df_train, rescaled_dim)

In [11]:
print(x_train.shape[1:])

(1000L,)


In [15]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

number_of_samples = x_train.shape[0]
one_tenth = int(number_of_samples * 0.10)
                     
x_train_fc, x_valid_fc = x_train[:one_tenth*8], x_train[one_tenth*8:one_tenth*9]
y_train_fc, y_valid_fc = y_train[:one_tenth*8], y_train[one_tenth*8:one_tenth*9]

if model_name == 'vgg16':
    model = custom_fc_layers(x_train.shape[1:], num_classes=17)
elif model_name == 'vgg19':
    model = vgg19_custom_top_classifier(x_train.shape[1:], num_classes=17)
elif model_name == 'resnet50':
    model = resnet50_custom_top_classifier(x_train.shape[1:], num_classes=17)
elif model_name == 'densenet121':
    model = densenet121_custom_top_classifier(x_train.shape[1:], num_classes=17)
elif model_name == 'inceptionv3':
    model = inceptionv3_custom_top_classifier(x_train.shape[1:], num_classes=17)
else:
    raise ValueError('ERROR: unsupport model {}'.format(model_name))

# early stopping prevents overfitting on training data
early_stop = EarlyStopping(monitor='val_loss', patience=3, min_delta=0, verbose=0, mode='auto')

file_uuid = time.strftime("%Y%m%d-%H%M%S") 
model_filepath = data_dir + 'bottleneck/{}/bottleneck_fc_model.h5'.format(model_name)
# save only the best model, not the latest epoch model.
checkpoint = ModelCheckpoint(model_filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

adam = optimizers.Adam(lr=0.0001)

model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy', 'recall', 'precision'])

history = model.fit(x_train_fc, y_train_fc,
                    nb_epoch=90,
                    batch_size=64,
                    verbose=1,
                    validation_data=(x_valid_fc, y_valid_fc),
                    callbacks=[checkpoint, early_stop])


Train on 32376 samples, validate on 4047 samples
Epoch 1/90
32376/32376 [==============================] - 1s - loss: 0.3011 - acc: 0.9074 - recall: 0.5904 - precision: 0.8187 - val_loss: 0.2054 - val_acc: 0.9222 - val_recall: 0.6663 - val_precision: 0.8432
Epoch 2/90
32376/32376 [==============================] - 1s - loss: 0.1901 - acc: 0.9278 - recall: 0.6923 - precision: 0.8531 - val_loss: 0.1788 - val_acc: 0.9317 - val_recall: 0.6948 - val_precision: 0.8781
Epoch 3/90
32376/32376 [==============================] - 1s - loss: 0.1711 - acc: 0.9350 - recall: 0.7169 - precision: 0.8763 - val_loss: 0.1663 - val_acc: 0.9364 - val_recall: 0.7140 - val_precision: 0.8900
Epoch 4/90
32376/32376 [==============================]

In [13]:
figures_dir = 'figures/{}/Bottleneck'.format(model_name)
makedirs(figures_dir)

plot_file_path = figures_dir + '/top_classifier_' + file_uuid + '.png'
trainHistoryPlot(plot_file_path, history.history, [], None)

In [14]:
from sklearn.metrics import fbeta_score, precision_score, recall_score

p_valid = model.predict(x_valid_fc, x_valid_fc.shape[0])

y_predictions = (np.array(p_valid) > 0.2).astype(int)

precision_s = precision_score(y_valid_fc, y_predictions, average='samples')
print('>>>> Overall precision score over validation set ' , precision_s)

recall_s = recall_score(y_valid_fc, y_predictions, average='samples')
print('>>>> Overall recall score over validation set ' , recall_s)

f2_score = fbeta_score(y_valid_fc, y_predictions, beta=2, average='samples')
print('>>>> Overall F2 score over validation set ' , f2_score)

('>>>> Overall precision score over validation set ', 0.82764530653707824)
('>>>> Overall recall score over validation set ', 0.92493469589466626)
('>>>> Overall F2 score over validation set ', 0.89212287257743816)
